### Generating Quadrotor EoMs using Kane's Method and Sympy

![Drone image](imgs/drone.PNG)

Modeling a CrazyFlie 2.0 Drone 
Using Project guide from Temple University MEE 411/5411:
https://github.com/BAmercury/MobileRobotics_Project2/blob/1a99b41272352da760fc6c3ba6914aceb616297c/Project2-Phase1.pdf
Thanks to Dr. Phillip Dames

#### Given:
mass: m = 0.030 kg
Distance from CoM to axis of a motor: L = 0.046 m
Inertia in the body-fixed frame:
![Drone inertia matrix](imgs/drone_inertia.PNG)

#### Motor Model:

Each rotor has an angular speed $\omega$<sub>i</sub> and produces a vertical force F<sub>i</sub> according to this simple model:

F<sub>i</sub> = k<sub>F</sub> * $\omega$<sub>i</sub><sup>2</sup>

k<sub>F</sub> ~= 6.11 * 10<sup>-8</sup>N/rpm<sup>2</sup>

Rotors also produce a moment according to:

M<sub>i</sub> = k<sub>M</sub> * $\omega$<sub>i</sub><sup>2</sup>

with k<sub>M</sub> = 1.5 * 10<sup>-9</sup>Nm/rpm<sup>2</sup>

Through system ID, rotor speed can be related to commanded speed through this first order ODE:

$\dot{w}$ = k<sub>m</sub> * ($\omega$<sub>i</sub><sup>des</sup> - $\omega$<sub>i</sub>)

Motor gain k<sub>m</sub> was found to be 20s<sup>-1</sup> by matching performance of the simulation with the real system



In [7]:
from sympy.physics.mechanics import *
from sympy import symbols, trigsimp
from numpy import deg2rad, rad2deg, array, zeros, linspace, pi
from scipy.integrate import odeint
from sympy.functions.elementary.trigonometric import sin, cos
from sympy import Matrix
from pydy.codegen.ode_function_generators import generate_ode_function

from sympy.physics.vector import init_vprinting
import matplotlib.pyplot as plt


In [57]:
# Define state vector

# Roll Pitch Yaw Euler angles
theta, phi, psi = dynamicsymbols('theta phi psi') # drone orientation expressed in inertial frame
thetad, phid, psid = dynamicsymbols('theta phi psi', 1) # drone angular velocity expressed in inertial frame
p, q, r = dynamicsymbols('p q r') # drone angular velocity in the body frame
# X Y Z translation
x, y, z = dynamicsymbols('x y z')
xd, yd, zd = dynamicsymbols('x y z', 1)

# Generalized speeds
# u1 = xd
# u2 = yd
# u3 = zd
# u4 = phid
# u5 = thetad
# u6 = psid
u1, u2, u3, u4, u5, u6 = dynamicsymbols('u1 u2 u3 u4 u5 u6')
u1d, u2d, u3d, u4d, u5d, u6d = dynamicsymbols('u1 u2 u3 u4 u5 u6', 1)

# Define Reference Frames
Nrf = ReferenceFrame('Nrf') # Inertial Reference Frame
# Body Fixed Reference Frame
# Z-X-Y Euler rotation, will write the DCM directly for sympy
cpsi = cos(psi)
spsi = sin(psi)
ctheta = cos(theta)
stheta = sin(theta)
cphi = cos(phi)
sphi = sin(phi)

c11 = cpsi*ctheta - sphi*spsi*stheta
c12 = -cphi*spsi
c13 = cpsi*stheta + ctheta*spsi*sphi
c21 = ctheta*spsi + cpsi*sphi*stheta
c22 = cphi*cpsi
c23 = spsi*stheta - cpsi*ctheta*sphi
c31 = -cphi*stheta
c32 = sphi
c33 = cphi*ctheta

BrfToNrf = Matrix([[c11, c12, c13],
                   [c21, c22, c23],
                   [c31, c32, c33]])
    

# Body frame is attached to CoM of drone
# b1 coincides with forward direction
# b3 is perpendicular ot plane of rotors pointing vertically up during hover
# body frame is parallel to principal axes
Brf = Nrf.orientnew('Brf', 'DCM', BrfToNrf.transpose())

# Set angular velocity of these frames
Brf.set_ang_vel(Brf, p*Brf.x + q*Brf.y + r*Brf.z)

# Define Locations
O = Point('O') # Inertial origin
O.set_vel(Nrf, 0)

# CoM of Drone
d_CG = Point('d_CG')
# Set the CG position and velocity
d_CG.set_pos(O, x*Nrf.x + y*Nrf.y + z*Nrf.z)
d_CG.set_vel(Nrf, d_CG.pos_from(O).dt(Nrf)) # Translation velocity
# Location of rotor motors
l = symbols('l') # length between drone cg and motor location
M1 = d_CG.locatenew('M1', l*Brf.x)
M2 = d_CG.locatenew('M2', l*Brf.y)
M3 = d_CG.locatenew('M3', -l*Brf.x)
M4 = d_CG.locatenew('M4', -l*Brf.y)

M1.v2pt_theory(d_CG, Nrf, Brf)
M2.v2pt_theory(d_CG, Nrf, Brf)
M3.v2pt_theory(d_CG, Nrf, Brf)
M4.v2pt_theory(d_CG, Nrf, Brf)

# Gather everything into lists
# X Y Z translation
coordinates = [x, y, z, phi, theta, psi]
speeds = [u1, u2, u3, u4, u5, u6]

# Kinematic Differential Equations
# u1 = xd
# u2 = yd
# u3 = zd
# u4 = phid
# u5 = thetad
# u6 = psid
kde = [u1 - xd, u2 - yd, u3 - zd, u4 - phid, u5 - thetad, u6 - psid]

# initial kane's object
kane = KanesMethod(Nrf, coordinates, speeds, kde)

# inertia dynamics
# inertia(N, 1, 2, 3, 4, 5, 6).to_matrix(N)
# Matrix([
#[1, 4, 6],
#[4, 2, 5],
#[6, 5, 3]])
I_d, m_drone = symbols('I_D m_drone') # Inertia of drone and mass 
itd = inertia(Brf, 1.43e-5, 1.43e-5, 2.89e-5, 0, 0, 0)

i1 = (itd, d_CG)

itd.to_matrix(Brf)

# Define rigid body objects
drone_body = RigidBody('drone_body', d_CG, Brf, m_drone, i1)

# Define forces and point of application
# Gravity
g = symbols('g') # gravity
# Thrust and Torque from each motor 
ThrM1, ThrM2, ThrM3, ThrM4, TorM1, TorM2, TorM3, TorM4 = symbols('ThrM1 ThrM2 ThrM3 ThrM4 TorM1 TorM2 TorM3 TorM4')
f_g = (d_CG, m_drone*g*Nrf.z)
f_M1 = (M1, 



Matrix([
[1.43e-5,       0,       0],
[      0, 1.43e-5,       0],
[      0,       0, 2.89e-5]])

In [58]:
# Should return Euler's Kinematical Equatoins
# [p, q, r] = H * [phi_d, theta_d, psi_d]
Nrf.ang_vel_in(Brf)

(-sin(theta(t))*cos(phi(t))*Derivative(psi(t), t) + cos(theta(t))*Derivative(phi(t), t))*Nrf.x + (sin(phi(t))*Derivative(psi(t), t) + Derivative(theta(t), t))*Nrf.y + (sin(theta(t))*Derivative(phi(t), t) + cos(phi(t))*cos(theta(t))*Derivative(psi(t), t))*Nrf.z

In [59]:
# Returns Brf to Nrf DCM
Brf.dcm(Nrf)

Matrix([
[-sin(phi(t))*sin(psi(t))*sin(theta(t)) + cos(psi(t))*cos(theta(t)), -sin(psi(t))*cos(phi(t)),  sin(phi(t))*sin(psi(t))*cos(theta(t)) + sin(theta(t))*cos(psi(t))],
[ sin(phi(t))*sin(theta(t))*cos(psi(t)) + sin(psi(t))*cos(theta(t)),  cos(phi(t))*cos(psi(t)), -sin(phi(t))*cos(psi(t))*cos(theta(t)) + sin(psi(t))*sin(theta(t))],
[                                        -sin(theta(t))*cos(phi(t)),              sin(phi(t)),                                          cos(phi(t))*cos(theta(t))]])

In [60]:
BrfToNrf

Matrix([
[-sin(phi(t))*sin(psi(t))*sin(theta(t)) + cos(psi(t))*cos(theta(t)), -sin(psi(t))*cos(phi(t)),  sin(phi(t))*sin(psi(t))*cos(theta(t)) + sin(theta(t))*cos(psi(t))],
[ sin(phi(t))*sin(theta(t))*cos(psi(t)) + sin(psi(t))*cos(theta(t)),  cos(phi(t))*cos(psi(t)), -sin(phi(t))*cos(psi(t))*cos(theta(t)) + sin(psi(t))*sin(theta(t))],
[                                        -sin(theta(t))*cos(phi(t)),              sin(phi(t)),                                          cos(phi(t))*cos(theta(t))]])